## Health Insurance Issuers by County--Republican states offer fewer options

Using the Bloomberg data from this story: https://www.bloomberg.com/graphics/2017-health-insurer-exits/ matched with population data from Census and the Governor's party affiliation from the NGA.

In [1]:
import pandas as pd

#### Collect data

In [4]:
df = pd.read_csv('issuers_bloomberg_data.csv').query('year == 2017').set_index('year')
df = df.groupby('fips_county').issuer_name.count()

df2 = pd.read_csv('census_pop.csv').set_index('state').join(
    pd.read_csv('nga_gov_list.csv').set_index('state')).dropna().reset_index().set_index('fips_county')
data = df2.join(df).dropna()

In [5]:
data.to_csv('clean_data.csv')

#### Basic calculations

In [6]:
data['issuer_number'] = data['issuer_name']
d = {1: '1', 2: '2', 3: '3+', 4: '3+', 5: '3+', 6: '3+', 7: '3+', 8: '3+', 9: '3+', 10: '3+', 11: '3+', 12: '3+', 13: '3+'}
data = data.replace({'issuer_number': d})

In [7]:
pd.pivot_table(data, values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

pop
gov_party   issuer_number           
Democratic  1               10714794
            2               24084228
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

In [8]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)

pop
gov_party   issuer_number           
Democratic  1                2114291
            2               22537943
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

In [9]:
pd.pivot_table(data, values='pop', index=['issuer_number'], aggfunc=sum)

,pop
issuer_number,
1,51630721
2,64847303
3+,205945767


In [10]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum).groupby(level=0).transform(lambda x: x/x.sum())

pop
gov_party   issuer_number          
Democratic  1              0.018040
            2              0.192300
            3+             0.789660
Independent 1              1.000000
Republican  1              0.206761
            2              0.209750
            3+             0.583489

In [11]:
len(data['state'].unique())

50

In [12]:
pd.pivot_table(data[~data['state'].str.contains('NC')], values='pop', index=['gov_party', 'issuer_number'], aggfunc=sum)#.groupby(level=0).transform(lambda x: x/x.sum())

pop
gov_party   issuer_number           
Democratic  1                2114291
            2               22537943
            3+              92549769
Independent 1                 733845
Republican  1               40182082
            2               40763075
            3+             113395998

In [43]:
ll = pd.read_table('2015_Gaz_counties_national.txt').rename(columns={'GEOID': 'fips_county'}).set_index('fips_county')
ll['INTPTLONG'] = ll.iloc[:,-1].rename('INTPTLONG')
ll = ll[['INTPTLAT', 'INTPTLONG']]

In [45]:
data = data.join(ll)

In [46]:
data.to_csv('clean_data.csv')

In [47]:
data

,state,county_name,pop,gov_name,gov_party,issuer_name,issuer_number,INTPTLAT,INTPTLONG
fips_county,,,,,,,,,
2013,AK,"Aleutians East Borough, Alaska",3296,Bill Walker,Independent,1.0,1,55.245044,-161.997477
2016,AK,"Aleutians West Census Area, Alaska",5647,Bill Walker,Independent,1.0,1,51.959447,178.338813
2020,AK,"Anchorage Municipality, Alaska",298192,Bill Walker,Independent,1.0,1,61.174250,-149.284329
2050,AK,"Bethel Census Area, Alaska",17968,Bill Walker,Independent,1.0,1,60.929141,-160.152625
2060,AK,"Bristol Bay Borough, Alaska",898,Bill Walker,Independent,1.0,1,58.730158,-156.996632
2068,AK,"Denali Borough, Alaska",1953,Bill Walker,Independent,1.0,1,63.681106,-150.026544
2070,AK,"Dillingham Census Area, Alaska",4954,Bill Walker,Independent,1.0,1,60.297320,-158.094726
2090,AK,"Fairbanks North Star Borough, Alaska",100605,Bill Walker,Independent,1.0,1,64.692317,-146.601733
2100,AK,"Haines Borough, Alaska",2496,Bill Walker,Independent,1.0,1,59.098771,-135.576936


In [11]:
tot = 51630721.0
d_sh = 10714794.0/tot
r_sh = 40182082.0/tot
tot_nc = 43030218.0
d_sh_nc = 10714794.0/tot_nc
r_sh_nc = 40182082.0/tot_nc

In [12]:
d_sh_nc

0.24900626810675233